In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/all/csv/places-of-all.csv")

In [3]:
df_no_duplicates = df.drop_duplicates()

In [4]:
df_no_duplicates

,place_id,name,main_category,phone,address
0,ChIJcWJ70U6rNTERxFCKqdDKXrw,"Joily Coffee, Tea & Bakery",Quán cà phê,0703 666 668,"Joily Coffee, Tea & Bakery, A10 Ng. Đức Hạnh 3..."
1,ChIJKbnmWQApdTERwsotmjhioIE,Du Miên Garden Cafe,Quán cà phê,028 3894 5555,"Du Miên Garden Cafe, 7 Đ. Phan Văn Trị, Phường..."
13,ChIJbUggYwcpdTERceZS9v1RSFY,Country Houses Coffee,Quán cà phê,0903 704 399,"Country Houses Coffee, 18 Đ. Phan Văn Trị, Phư..."
23,ChIJ32cXs5erNTERsqdVf0cP-mg,Xofa Café & Bistro,Quán cà phê,024 3717 1555,"Xofa Café & Bistro, 14 P. Tống Duy Tân, Hàng B..."
25,ChIJiXZG2IqrNTERdSRq4nRh9U4,"Lofita - Chuỗi quán Trà & Cà Phê (Cafe) Ngon, ...",Quán cà phê,1800 6503,"Lofita - Chuỗi quán Trà & Cà Phê (Cafe) Ngon, ..."
...,...,...,...,...,...
2413,ChIJ-WnxKyaJoDERR6rIzeW_WdE,Dehavi Coffee,Quán cà phê,NaN,"2P8P+7FM Dehavi Coffee, Nguyễn Văn Cừ Nối Dài,..."
2414,ChIJ5V0eWDdjoDERrTBDf1nIGzY,Take Away Coffee ( HẠ ),Quán cà phê,0942 819 113,"Take Away Coffee ( HẠ ), 46 Đ. Trương Định, Tâ..."
2415,ChIJBf8k86BioDERG2hx89DdiJ8,Cà Phê Minh Thanh,Quán cà phê,0292 3821 147,"Cà Phê Minh Thanh, 91 Đ. Ngô Quyền, Tân An, Ni..."
2416,ChIJaVP7kKBioDERXg7hXg2zXPQ,Cà Phê Mode,Quán cà phê,0292 3819 141,"Cà Phê Mode, 6 Hẻm 22 Đ. Mạc Đĩnh Chi, Tân An,..."


In [12]:
import pandas as pd

# Tạo DataFrame mẫu
data = {'name': ['Joily Coffee, Tea & Bakery'],
        'address': ['Joily Coffee, Tea & Bakery, A10 Ng. Đức Hạnh 3']}

df = pd.DataFrame(data)

# Tách giá trị cột 'name' khỏi cột 'address'
df['address'] = df.apply(lambda row: row['address'].replace(row['name']+", ", '').strip(), axis=1)

# Hiển thị kết quả
print(df)



                         name             address
0  Joily Coffee, Tea & Bakery  A10 Ng. Đức Hạnh 3


In [13]:
df_no_duplicates['address'] = df_no_duplicates.apply(lambda row: row['address'].replace(row['name']+", ", '').strip(), axis=1)

In [14]:
import requests

def get_location(address):
    api_keys = ["mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M",
                "br4m9W3KMhrPa51Vru5M1mWJYCfaVFvWiP3kB2gEQUA",
                "spdVYGb-b6cgokZBOuwGG4b3WbmgsxQqv7tudjdOfLc",
                "0zeR-GEwo5Od-poIZYTGrw-I80fC2C0b0q4F1Boa8oc",
                "UfHD68dH4O9ld-NS22Se6Bls98P9aPZhFq4hA_H7-74",
                "c6NrKs_BpAX4OdRFKAVSDeUaQTjSpNF8dCxjcLT6AfE",
                "WCpDyLiAACoL9zwSFQQa_HboP-HdWXSM601W1N02aeg"]
    current_key_index = 0
    api_key = api_keys[current_key_index]
    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key}'
    
    try:
        # Gửi yêu cầu API
        response = requests.get(url)
        
        # Nếu API key đã hết hạn, thử API key tiếp theo trong danh sách
        key_count = 0
        while response.status_code == 429:
            key_count += 1
            # Nếu thử hết key trả ra tập rỗng và progess là False đánh dấu những case chưa xử lí
            if key_count == len(api_keys):
                return {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
            current_key_index = current_key_index + 1
            api_key = api_keys[current_key_index]
            url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key}'
            response = requests.get(url)
        data = response.json()    
        # Xử lý kết quả và trả về
        if data == {'items': []}: 
          list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "True" }
        else: 
          result = data['items'][0]
          position = result['position']
          lat = position.get('lat', '')
          long = position.get('lng', '')
          address_info = result['address']
          city = address_info.get('county', '')
          district = address_info.get('city', '')
          ward = address_info.get('district', '')
          street = address_info.get('street', '')
          houseNumber = address_info.get('houseNumber', '')
          list_address = {"lat": lat, "lng": long, "House_number": houseNumber, "Street": street, "Ward": ward, "District": district, "City": city, "progress": "True" }
    except:
           # Nếu thử xảy ra lỗi trả ra tập rỗng và progess là False đánh dấu những case chưa xử lí
        list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "hCity": "", "progress": "False" }
           
    return list_address


In [20]:
df_no_duplicates.reset_index()

,index,place_id,name,main_category,phone,address
0,0,ChIJcWJ70U6rNTERxFCKqdDKXrw,"Joily Coffee, Tea & Bakery",Quán cà phê,0703 666 668,"A10 Ng. Đức Hạnh 3, Khu đô thị Nam Trung Yên, ..."
1,1,ChIJKbnmWQApdTERwsotmjhioIE,Du Miên Garden Cafe,Quán cà phê,028 3894 5555,"7 Đ. Phan Văn Trị, Phường 10, Gò Vấp, Thành ph..."
2,13,ChIJbUggYwcpdTERceZS9v1RSFY,Country Houses Coffee,Quán cà phê,0903 704 399,"18 Đ. Phan Văn Trị, Phường 10, Gò Vấp, Thành p..."
3,23,ChIJ32cXs5erNTERsqdVf0cP-mg,Xofa Café & Bistro,Quán cà phê,024 3717 1555,"14 P. Tống Duy Tân, Hàng Bông, Hoàn Kiếm, Hà Nội"
4,25,ChIJiXZG2IqrNTERdSRq4nRh9U4,"Lofita - Chuỗi quán Trà & Cà Phê (Cafe) Ngon, ...",Quán cà phê,1800 6503,"225 P. Trần Quốc Hoàn, Dịch Vọng Hậu, Cầu Giấy..."
...,...,...,...,...,...,...
730,2413,ChIJ-WnxKyaJoDERR6rIzeW_WdE,Dehavi Coffee,Quán cà phê,NaN,"2P8P+7FM Nguyễn Văn Cừ Nối Dài, An Bình, Ninh ..."
731,2414,ChIJ5V0eWDdjoDERrTBDf1nIGzY,Take Away Coffee ( HẠ ),Quán cà phê,0942 819 113,"46 Đ. Trương Định, Tân An, Ninh Kiều, Cần Thơ"
732,2415,ChIJBf8k86BioDERG2hx89DdiJ8,Cà Phê Minh Thanh,Quán cà phê,0292 3821 147,"91 Đ. Ngô Quyền, Tân An, Ninh Kiều, Cần Thơ"
733,2416,ChIJaVP7kKBioDERXg7hXg2zXPQ,Cà Phê Mode,Quán cà phê,0292 3819 141,"6 Hẻm 22 Đ. Mạc Đĩnh Chi, Tân An, Ninh Kiều, C..."


In [22]:
df_no_duplicates = df_no_duplicates.reset_index()

In [23]:
df_no_duplicates

,index,place_id,name,main_category,phone,address
0,0,ChIJcWJ70U6rNTERxFCKqdDKXrw,"Joily Coffee, Tea & Bakery",Quán cà phê,0703 666 668,"A10 Ng. Đức Hạnh 3, Khu đô thị Nam Trung Yên, ..."
1,1,ChIJKbnmWQApdTERwsotmjhioIE,Du Miên Garden Cafe,Quán cà phê,028 3894 5555,"7 Đ. Phan Văn Trị, Phường 10, Gò Vấp, Thành ph..."
2,13,ChIJbUggYwcpdTERceZS9v1RSFY,Country Houses Coffee,Quán cà phê,0903 704 399,"18 Đ. Phan Văn Trị, Phường 10, Gò Vấp, Thành p..."
3,23,ChIJ32cXs5erNTERsqdVf0cP-mg,Xofa Café & Bistro,Quán cà phê,024 3717 1555,"14 P. Tống Duy Tân, Hàng Bông, Hoàn Kiếm, Hà Nội"
4,25,ChIJiXZG2IqrNTERdSRq4nRh9U4,"Lofita - Chuỗi quán Trà & Cà Phê (Cafe) Ngon, ...",Quán cà phê,1800 6503,"225 P. Trần Quốc Hoàn, Dịch Vọng Hậu, Cầu Giấy..."
...,...,...,...,...,...,...
730,2413,ChIJ-WnxKyaJoDERR6rIzeW_WdE,Dehavi Coffee,Quán cà phê,NaN,"2P8P+7FM Nguyễn Văn Cừ Nối Dài, An Bình, Ninh ..."
731,2414,ChIJ5V0eWDdjoDERrTBDf1nIGzY,Take Away Coffee ( HẠ ),Quán cà phê,0942 819 113,"46 Đ. Trương Định, Tân An, Ninh Kiều, Cần Thơ"
732,2415,ChIJBf8k86BioDERG2hx89DdiJ8,Cà Phê Minh Thanh,Quán cà phê,0292 3821 147,"91 Đ. Ngô Quyền, Tân An, Ninh Kiều, Cần Thơ"
733,2416,ChIJaVP7kKBioDERXg7hXg2zXPQ,Cà Phê Mode,Quán cà phê,0292 3819 141,"6 Hẻm 22 Đ. Mạc Đĩnh Chi, Tân An, Ninh Kiều, C..."


In [15]:
# Đọc file đầu vào
df2 = df_no_duplicates
# Lặp qua tất cả address
address_info_list = []
for address in df2["address"]:
    address_info = get_location(address)
    address_info_list.append(address_info)

# Tạo ra một DataFrame mới từ list này và thêm vào DataFrame gốc
address_df = pd.DataFrame(address_info_list)
df10 = pd.concat([df2, address_df], axis=1)

In [16]:
df10

,place_id,name,main_category,phone,address,lat,lng,House_number,Street,Ward,District,City,progress
0,ChIJcWJ70U6rNTERxFCKqdDKXrw,"Joily Coffee, Tea & Bakery",Quán cà phê,0703 666 668,"A10 Ng. Đức Hạnh 3, Khu đô thị Nam Trung Yên, ...",21.01752,105.78672,AR7,Khu Đô Thị Nam Trung Yên,Phường Yên Hòa,Quận Cầu Giấy,Hà Nội,True
1,ChIJKbnmWQApdTERwsotmjhioIE,Du Miên Garden Cafe,Quán cà phê,028 3894 5555,"7 Đ. Phan Văn Trị, Phường 10, Gò Vấp, Thành ph...",10.83195,106.67418,7,Đường Phan Văn Trị,Phường 10,Quận Gò Vấp,Hồ Chí Minh,True
13,ChIJbUggYwcpdTERceZS9v1RSFY,Country Houses Coffee,Quán cà phê,0903 704 399,"18 Đ. Phan Văn Trị, Phường 10, Gò Vấp, Thành p...",10.77402,106.70419,42,Đường Nguyễn Huệ,Phường Bến Nghé,Quận 1,Hồ Chí Minh,True
23,ChIJ32cXs5erNTERsqdVf0cP-mg,Xofa Café & Bistro,Quán cà phê,024 3717 1555,"14 P. Tống Duy Tân, Hàng Bông, Hoàn Kiếm, Hà Nội",10.02463,105.76727,49,Đường 3 Tháng 2,Phường Xuân Khánh,Quận Ninh Kiều,Cần Thơ,True
25,ChIJiXZG2IqrNTERdSRq4nRh9U4,"Lofita - Chuỗi quán Trà & Cà Phê (Cafe) Ngon, ...",Quán cà phê,1800 6503,"225 P. Trần Quốc Hoàn, Dịch Vọng Hậu, Cầu Giấy...",10.0339,105.78282,42,Đường Đề Thám,Phường An Cư,Quận Ninh Kiều,Cần Thơ,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,NaN,NaN,NaN,NaN,NaN,10.05044,105.77253,,Nguyễn Văn Cừ,Phường An Hòa,Quận Ninh Kiều,Cần Thơ,True
730,NaN,NaN,NaN,NaN,NaN,10.02731,105.74882,7,Đường Nguyễn Văn Cừ,Phường An Bình,Quận Ninh Kiều,Cần Thơ,True
732,NaN,NaN,NaN,NaN,NaN,10.03584,105.78351,91,Đường Ngô Quyền,Phường An Cư,Quận Ninh Kiều,Cần Thơ,True
733,NaN,NaN,NaN,NaN,NaN,10.03657,105.78243,6,Hẻm 22 Mạc Đĩnh Chi,Phường An Cư,Quận Ninh Kiều,Cần Thơ,True


In [8]:
df10.to_csv("caphe.csv")

In [9]:
df_no_duplicates.to_csv("df_no_duplicates.csv")

In [31]:
df_bar = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-bar/csv/places-of-quán-bar.csv")

In [37]:
df_bar = df_bar.drop_duplicates()

In [48]:
df_bar.to_csv("df_bar.csv")

In [32]:
df_bida = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/places-of-quán-bida.csv")

In [38]:
df_bida = df_bida.drop_duplicates()

In [49]:
df_bida.to_csv("df_bida.csv")

In [45]:
df_quan_an = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/places-of-quán-ăn.csv")

In [46]:
df_quan_an = df_quan_an.drop_duplicates()

In [50]:
df_quan_an.to_csv("df_quan_an.csv")

In [47]:
df_quan_an

,place_id,name,main_category,categories,phone,address,address_ward,address_street,address_city,address_postal_code,address_state,address_country_code
0,ChIJLTbDq69ioDERm2YShtUfRic,Nhà Hàng Lúa Nếp,Nhà hàng,Nhà hàng,+84 292 3676 979,"2QVV+WM5 Nhà Hàng Lúa Nếp, Khu bãi bồi, p. Cái...","Cái Khế, Ninh Kiều","2QVV+WM5, Khu bãi bồi, p. Cái Khế, Lê Lợi",Cần Thơ,NaN,"Ninh Kiều, Cần Thơ",VN
3,ChIJ2ZJaF12IoDERnzIHDfrFnb8,Quán Bánh xèo 7 Tới,Quán ăn nhỏ,Quán ăn nhỏ,+84 947 699 743,"Quán Bánh xèo 7 Tới, 45 Đ. Hoàng Quốc Việt, An...","An Bình, Ninh Kiều",45 Đ. Hoàng Quốc Việt,Cần Thơ,NaN,"Ninh Kiều, Cần Thơ",VN
7,ChIJJcVmoh-IoDEREXEUXrzGpbU,Quán Lẩu Vịt Nấu Chao Thành Giao,Nhà hàng,"Nhà hàng, Quán ăn nhỏ",+84 939 933 949,"Quán Lẩu Vịt Nấu Chao Thành Giao, hẻm 1 Đ. Lý ...","An Phú, Ninh Kiều",hẻm 1 Đ. Lý Tự Trọng,Cần Thơ,NaN,"Ninh Kiều, Cần Thơ",VN
10,ChIJk0ML7KRioDERxaR-kkPZEBw,Nhà hàng Hoa Sứ,Nhà hàng,Nhà hàng,+84 292 3820 717,"2QQR+4JV Nhà hàng Hoa Sứ, KDL, Ninh Kiều, Cần ...","Cái Khế, Ninh Kiều","2QQR+4JV, KDL",Cần Thơ,NaN,"Ninh Kiều, Cần Thơ",VN
13,ChIJy8wPWKhioDER2w3ad4TlCCQ,Cơm Gà 6 Quận,Quán ăn nhỏ,Quán ăn nhỏ,+84 908 816 188,"Cơm Gà 6 Quận, 109 Trần Văn Khéo, Cái Khế, Nin...","Cái Khế, Ninh Kiều",109 Trần Văn Khéo,Cần Thơ,NaN,"Ninh Kiều, Cần Thơ",VN
...,...,...,...,...,...,...,...,...,...,...,...,...
1913,ChIJ4XFWbRljoDERejO6PdaQiSo,Quán Gà Ác Toàn,Nhà hàng gia đình,Nhà hàng gia đình,+84 292 3821 968,"2QPM+XCV Quán Gà Ác Toàn, Hẻm 9 Đ. Xô Viết Ngh...","An Cư, Ninh Kiều","2QPM+XCV, Hẻm 9 Đ. Xô Viết Nghệ Tĩnh",Cần Thơ,NaN,"Ninh Kiều, Cần Thơ",VN
1914,ChIJid1kOOaJoDERlm-U5x9aKQE,Quán ốc Vĩa Hè 2,Quán ăn nhỏ,Quán ăn nhỏ,+84 796 851 131,"2QMH+6RF Quán ốc Vĩa Hè 2, Đ. Lê Lai, An Lạc, ...","An Lạc, Ninh Kiều","2QMH+6RF, Đ. Lê Lai",Cần Thơ,NaN,"Ninh Kiều, Cần Thơ",VN
1916,ChIJkxP5PKBioDERURCwhAH3tGo,Quán Cơm Chay Lâm Thị Hai,Gian hàng rong,"Gian hàng rong, Quán ăn tự phục vụ",+84 937 051 148,"Quán Cơm Chay Lâm Thị Hai, 42, Đường Hòa Bình,...","Tân An, Ninh Kiều","42, Đường Hòa Bình, Phường An Cư, Quận Ninh Kiều",Cần Thơ,NaN,"Ninh Kiều, Cần Thơ",VN
1917,ChIJ-0X_FJ9ioDERB6tVSQyDAY0,Quán Cơm 115,Nhà hàng,Nhà hàng,+84 292 3834 462,"Quán Cơm 115, 115, Châu Văn Liêm, Quận Ninh Ki...","Tân An, Ninh Kiều","115, Châu Văn Liêm, Quận Ninh Kiều, Tp Cần Thơ",Cần Thơ,NaN,"Ninh Kiều, Cần Thơ",VN


In [34]:
df_ca_phe = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-cà-phê/csv/places-of-quán-cà-phê.csv")

In [40]:
df_ca_phe = df_ca_phe.drop_duplicates()

In [51]:
df_ca_phe.to_csv("df_ca_phe.csv")

In [35]:
df_karaoke = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-karaoke/csv/places-of-quán-karaoke.csv")

In [41]:
df_karaoke = df_karaoke.drop_duplicates()

In [52]:
df_karaoke.to_csv("df_karaoke.csv")

In [53]:
df_buffet = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-nước/csv/places-of-quán-nước.csv")

In [54]:
df_buffet = df_buffet.drop_duplicates()

In [55]:
df_buffet.to_csv("df_buffet.csv")

In [56]:
df_nuoc = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-nước/csv/places-of-quán-nước.csv")

In [57]:
df_nuoc = df_nuoc.drop_duplicates()

In [58]:
df_nuoc.to_csv("df_nuoc.csv")

In [61]:
df_bun_pho = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-bún-phở/csv/places-of-quán-bún-phở.csv")

In [62]:
df_bun_pho = df_bun_pho.drop_duplicates()

In [64]:
df_bun_pho.to_csv("df_bun_pho.csv")

In [66]:
df_lau_nuong = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-lẩu-nướng/csv/places-of-quán-lẩu-nướng.csv")

In [67]:
df_lau_nuong = df_lau_nuong.drop_duplicates()

In [71]:
df_lau_nuong.to_csv("df_lau_nuong.csv")

In [72]:
df_nha_hang = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/nhà-hàng/csv/places-of-nhà-hàng.csv")

In [73]:
df_nha_hang = df_nha_hang.drop_duplicates()

In [77]:
df_bun = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-bún/csv/places-of-quán-bún.csv")

In [78]:
df_bun = df_bun.drop_duplicates()

In [79]:
df_com = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-cơm/csv/places-of-quán-cơm.csv")

In [80]:
df_com = df_com.drop_duplicates()

In [81]:
df_pho = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-phở/csv/places-of-quán-phở.csv").drop_duplicates()

In [82]:
df_nuong = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/places-of-quán-nướng.csv").drop_duplicates()

In [83]:
df_lau = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/places-of-quán-lẩu.csv").drop_duplicates()

In [84]:
df_chao = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-cháo/csv/places-of-quán-cháo.csv").drop_duplicates()

In [85]:
df_bia = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-bia/csv/places-of-quán-bia.csv").drop_duplicates()

In [86]:
df_nhau = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-nhậu/csv/places-of-quán-nhậu.csv").drop_duplicates()

In [89]:
df_banh = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-bánh/csv/places-of-quán-bánh.csv").drop_duplicates()

In [92]:
df_tra = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/quán-trà/csv/places-of-quán-trà.csv").drop_duplicates()

In [87]:
df_fnb_ninh_kieu = pd.concat([df_bar, df_bida, df_quan_an, df_ca_phe, df_karaoke, df_buffet, df_nuoc, df_bun_pho, df_lau_nuong, df_nha_hang, df_bun, df_com, df_pho, df_nuong, df_lau, df_chao, df_bia, df_nhau ], axis=0, ignore_index=True)

In [93]:
df_fnb_ninh_kieu = pd.concat([df_fnb_ninh_kieu, df_tra], axis=0, ignore_index=True)

In [96]:
df_fnb_ninh_kieu.to_excel("df_fnb_ninh_kieu.xlsx")

In [95]:
df_fnb_ninh_kieu['address'] = df_fnb_ninh_kieu.apply(lambda row: row['address'].replace(row['name']+", ", '').strip(), axis=1)